In [1]:
import tensorflow as tf
import numpy as np
import os
import time
import datetime
import helper
from text_cnn import TextCNN
from tensorflow.contrib import learn
import math
import yaml

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected

# Parameters

In [2]:
# Data loading params
# FLAGS are command-line arguments to our program.
tf.flags.DEFINE_float("dev_sample_percentage", .1, "Percentage of the training data to use for validation")
#tf.flags.DEFINE_string("positive_data_file", "./data/rt-polaritydata/rt-polarity.pos", "Data source for the positive data.")
#tf.flags.DEFINE_string("negative_data_file", "./data/rt-polaritydata/rt-polarity.neg", "Data source for the negative data.")
tf.flags.DEFINE_string("positive_data_file", "./data/rt-polaritydata/rt-polarity.pos", "Data source for the positive data.")
tf.flags.DEFINE_string("negative_data_file", "./data/rt-polaritydata/rt-polarity.neg", "Data source for the negative data.")

# Model Hyperparameters
tf.flags.DEFINE_boolean("enable_word_embeddings", True, "Enable/disable the word embedding (default: True)")
tf.flags.DEFINE_integer("embedding_dim", 128, "Dimensionality of character embedding (default: 128)")
tf.flags.DEFINE_string("filter_sizes", "3,4,5", "Comma-separated filter sizes (default: '3,4,5')")
tf.flags.DEFINE_integer("num_filters", 128, "Number of filters per filter size (default: 128)")
tf.flags.DEFINE_float("dropout_keep_prob", 0.4, "Dropout keep probability (default: 0.5)")
tf.flags.DEFINE_float("l2_reg_lambda", 9.0, "L2 regularization lambda (default: 0.0)")


# Training parameters
tf.flags.DEFINE_integer("batch_size", 64, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("num_epochs", 30, "Number of training epochs (default: 200)")
tf.flags.DEFINE_integer("evaluate_every", 100, "Evaluate model on dev set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")

# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")
tf.flags.DEFINE_float("decay_coefficient", 2.5, "Decay coefficient (default: 2.5)")
tf.app.flags.DEFINE_string('f', '', 'kernel')
FLAGS = tf.flags.FLAGS

# load config file
with open("config.yml", 'r') as ymlfile:
    cfg = yaml.load(ymlfile)
    
dataset_name = cfg["datasets"]["default"]
if FLAGS.enable_word_embeddings and cfg['word_embeddings']['default'] is not None:
    embedding_name = cfg['word_embeddings']['default']
    embedding_dimension = cfg['word_embeddings'][embedding_name]['dimension']
else:
    embedding_dimension = FLAGS.embedding_dim




# Data preprocessing

In [3]:
def preprocess():

    # Load data
    print("Loading data...")
    #x_text, y = helper.load_data_and_labels(FLAGS.positive_data_file, FLAGS.negative_data_file)
    
    datasets = None
    if dataset_name == "mrpolarity":
        datasets = helper.get_datasets_mrpolarity(cfg["datasets"][dataset_name]["positive_data_file"]["path"],
                                                        cfg["datasets"][dataset_name]["negative_data_file"]["path"])
    elif dataset_name == "20newsgroup":
        datasets = helper.get_datasets_20newsgroup(subset="train", 
                                                         categories=cfg["datasets"][dataset_name]["categories"],
                                                         shuffle=cfg["datasets"][dataset_name]["shuffle"],
                                                         random_state=cfg["datasets"][dataset_name]["random_state"])
    elif dataset_name == "localdata":
        datasets = helper.get_datasets_localdata(container_path=cfg["datasets"][dataset_name]["container_path"],
                                                         categories=cfg["datasets"][dataset_name]["categories"],
                                                         shuffle=cfg["datasets"][dataset_name]["shuffle"],
                                                         random_state=cfg["datasets"][dataset_name]["random_state"])
    x_text, y = helper.load_data_labels(datasets)

    # Build vocabulary
    max_document_length = max([len(x.split(" ")) for x in x_text])
    vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
    x = np.array(list(vocab_processor.fit_transform(x_text)))

    # Randomly shuffle data
    np.random.seed(10)
    shuffle_indices = np.random.permutation(np.arange(len(y)))
    x_shuffled = x[shuffle_indices]
    y_shuffled = y[shuffle_indices]

    # Split train/test set
    # TODO: This is very crude, should use cross-validation
    dev_sample_index = -1 * int(FLAGS.dev_sample_percentage * float(len(y)))
    x_train, x_dev = x_shuffled[:dev_sample_index], x_shuffled[dev_sample_index:]
    y_train, y_dev = y_shuffled[:dev_sample_index], y_shuffled[dev_sample_index:]

    del x, y, x_shuffled, y_shuffled

    print("Vocabulary Size: {:d}".format(len(vocab_processor.vocabulary_)))
    print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))
    return x_train, y_train, vocab_processor, x_dev, y_dev

# Training

In [4]:
def train(x_train, y_train, vocab_processor, x_dev, y_dev):
    # Training
    # ==================================================
    graph = tf.Graph()
    gpu_options=tf.GPUOptions()
    gpu_options.allow_growth = True
    with graph.as_default():
        session_conf = tf.ConfigProto(
          allow_soft_placement=FLAGS.allow_soft_placement,
          log_device_placement=FLAGS.log_device_placement,
          gpu_options=gpu_options)

        sess = tf.Session(config= session_conf)
        with sess.as_default():
            cnn = TextCNN(
                sequence_length= x_train.shape[1],
                num_classes= y_train.shape[1],
                vocab_size= len(vocab_processor.vocabulary_),
                #embedding_size=FLAGS.embedding_dim,
                embedding_size= embedding_dimension,
                filter_sizes= list(map(int, FLAGS.filter_sizes.split(","))),
                num_filters= FLAGS.num_filters,
                l2_reg_lambda= FLAGS.l2_reg_lambda)

            # Define Training procedure
            global_step = tf.Variable(0, name="global_step", trainable=False)

            optimizer = tf.train.AdamOptimizer(cnn.learning_rate)
            grads_and_vars = optimizer.compute_gradients(cnn.loss)
            train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)
            

            # Keep track of gradient values and sparsity (optional)
            grad_summaries = []
            for g, v in grads_and_vars:
                if g is not None:
                    grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                    sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                    grad_summaries.append(grad_hist_summary)
                    grad_summaries.append(sparsity_summary)
            grad_summaries_merged = tf.summary.merge(grad_summaries)
            
             # Output directory for models and summaries
            timestamp = str(int(time.time()))
            out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
            print("Writing to {}\n".format(out_dir))

            # Summaries for loss and accuracy
            loss_summary = tf.summary.scalar("loss", cnn.loss)
            acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)
        
            # Train Summaries
            train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
            train_summary_dir = os.path.join(out_dir, "summaries", "train")
            train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

            # Dev summaries'
            dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
            dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
            dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

            # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
            checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
            checkpoint_prefix = os.path.join(checkpoint_dir, "model")
            if not os.path.exists(checkpoint_dir):
                os.makedirs(checkpoint_dir)
            saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

            # Write vocabulary
            vocab_processor.save(os.path.join(out_dir, "vocab"))

            # Initialize all variables
            sess.run(tf.global_variables_initializer())
            
            # Set the embedding
            if FLAGS.enable_word_embeddings and cfg['word_embeddings']['default'] is not None:
                vocabulary = vocab_processor.vocabulary_
                initW = None
                if embedding_name == 'word2vec':
                    # load embedding vectors from the word2vec
                    print("Load word2vec file {}".format(cfg['word_embeddings']['word2vec']['path']))
                    initW = helper.load_embedding_vectors_word2vec(vocabulary,
                                                                         cfg['word_embeddings']['word2vec']['path'],
                                                                         cfg['word_embeddings']['word2vec']['binary'])
                    print("word2vec file has been loaded")
                elif embedding_name == 'glove':
                    # load embedding vectors from the glove
                    print("Load glove file {}".format(cfg['word_embeddings']['glove']['path']))
                    initW = helper.load_embedding_vectors_glove(vocabulary,
                                                                      cfg['word_embeddings']['glove']['path'],
                                                                      embedding_dimension)
                    print("glove file has been loaded\n")

                sess.run(cnn.W.assign(initW))
            
            
            #def train_step(x_batch, y_batch):
            def train_step(x_batch, y_batch, learning_rate):
                """
                A single training step
                """
                feed_dict = {
                  cnn.input_x: x_batch,
                  cnn.input_y: y_batch,
                  cnn.dropout_keep_prob: FLAGS.dropout_keep_prob,
                  cnn.learning_rate: learning_rate
                }
                _, step, summaries, loss, accuracy= sess.run(
                    [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                    feed_dict)

               
                time_str = datetime.datetime.now().isoformat()
                print("{}: step {}, loss {:g}, acc {:g}, learning_rate {:g}" 
                      .format(time_str, step, loss, accuracy, learning_rate))
                
                train_summary_writer.add_summary(summaries, step)

            def dev_step(x_batch, y_batch, writer=None):
                """
                Evaluates model on a dev set
                """
                feed_dict = {
                  cnn.input_x: x_batch,
                  cnn.input_y: y_batch,
                  cnn.dropout_keep_prob: 1.0
                }
                step, summaries, loss, accuracy= sess.run(
                    [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                    feed_dict)
                time_str = datetime.datetime.now().isoformat()
                print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
                if writer:
                    writer.add_summary(summaries, step)
                    
            # Generate batches
            batches = helper.batch_iter(
                list(zip(x_train, y_train)), FLAGS.batch_size, FLAGS.num_epochs)
             # It uses dynamic learning rate with a high value at the beginning to speed up the training
            #max_learning_rate = 0.005
            #min_learning_rate = 0.0001
            max_learning_rate = 0.005
            min_learning_rate = 0.0001
            decay_speed = FLAGS.decay_coefficient*len(y_train)/FLAGS.batch_size
            # Training loop. For each batch...
            counter = 0
            for batch in batches:
                learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate) * math.exp(-counter/decay_speed)
                counter += 1
                x_batch, y_batch = zip(*batch)
                #train_step(x_batch, y_batch)
                train_step(x_batch, y_batch, learning_rate)
                current_step = tf.train.global_step(sess, global_step)
                
                if current_step % FLAGS.evaluate_every == 0:
                    print("\nEvaluation:")
                    dev_step(x_dev, y_dev, writer=dev_summary_writer)
                    print("")
                if current_step % FLAGS.checkpoint_every == 0:
                    path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                    print("Saved model checkpoint to {}\n".format(path))
                    
                    

In [5]:
def main(argv=None):
    x_train, y_train, vocab_processor, x_dev, y_dev = preprocess()
    train(x_train, y_train, vocab_processor, x_dev, y_dev)

In [6]:
if __name__ == '__main__':
    tf.app.run()

Loading data...
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Vocabulary Size: 18758
Train/Dev split: 9596/1066
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/filter_matrix:0/grad/hist is illegal; using conv-maxpool-3/filter_matrix_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/filter_matrix:0/grad/sparsity is illegal; using conv-maxpool-3/filter_matrix_0/grad/sparsity instead.
INFO:tensorflow:Summary name 

2019-01-29T02:23:31.246797: step 51, loss 0.748306, acc 0.625, learning_rate 0.00438811
2019-01-29T02:23:31.376475: step 52, loss 0.713672, acc 0.640625, learning_rate 0.00437669
2019-01-29T02:23:31.505947: step 53, loss 0.808519, acc 0.40625, learning_rate 0.00436529
2019-01-29T02:23:31.592396: step 54, loss 0.758749, acc 0.5, learning_rate 0.00435393
2019-01-29T02:23:31.712962: step 55, loss 0.751516, acc 0.5, learning_rate 0.0043426
2019-01-29T02:23:31.836430: step 56, loss 0.730365, acc 0.515625, learning_rate 0.00433129
2019-01-29T02:23:31.958955: step 57, loss 0.692783, acc 0.734375, learning_rate 0.00432002
2019-01-29T02:23:32.083924: step 58, loss 0.773755, acc 0.5, learning_rate 0.00430878
2019-01-29T02:23:32.214298: step 59, loss 0.80088, acc 0.484375, learning_rate 0.00429756
2019-01-29T02:23:32.314874: step 60, loss 0.705145, acc 0.625, learning_rate 0.00428638
2019-01-29T02:23:32.399505: step 61, loss 0.691355, acc 0.625, learning_rate 0.00427523
2019-01-29T02:23:32.498721

2019-01-29T02:23:42.693031: step 141, loss 0.601335, acc 0.703125, learning_rate 0.00347281
2019-01-29T02:23:42.823091: step 142, loss 0.534288, acc 0.765625, learning_rate 0.00346382
2019-01-29T02:23:42.948680: step 143, loss 0.626747, acc 0.703125, learning_rate 0.00345486
2019-01-29T02:23:43.080232: step 144, loss 0.566446, acc 0.765625, learning_rate 0.00344592
2019-01-29T02:23:43.212349: step 145, loss 0.553222, acc 0.75, learning_rate 0.00343701
2019-01-29T02:23:43.301728: step 146, loss 0.548275, acc 0.796875, learning_rate 0.00342812
2019-01-29T02:23:43.428921: step 147, loss 0.563537, acc 0.75, learning_rate 0.00341925
2019-01-29T02:23:43.557194: step 148, loss 0.570066, acc 0.75, learning_rate 0.00341041
2019-01-29T02:23:43.681350: step 149, loss 0.591084, acc 0.734375, learning_rate 0.00340159
2019-01-29T02:23:43.906152: step 150, loss 0.505481, acc 0.833333, learning_rate 0.00339279
2019-01-29T02:23:44.025025: step 151, loss 0.52145, acc 0.84375, learning_rate 0.00338402
20

2019-01-29T02:23:53.407419: step 230, loss 0.293404, acc 0.953125, learning_rate 0.00275997
2019-01-29T02:23:53.532529: step 231, loss 0.468007, acc 0.8125, learning_rate 0.00275288
2019-01-29T02:23:53.618584: step 232, loss 0.448845, acc 0.84375, learning_rate 0.00274581
2019-01-29T02:23:53.717664: step 233, loss 0.403467, acc 0.921875, learning_rate 0.00273876
2019-01-29T02:23:53.841982: step 234, loss 0.415909, acc 0.84375, learning_rate 0.00273173
2019-01-29T02:23:53.973085: step 235, loss 0.491647, acc 0.828125, learning_rate 0.00272472
2019-01-29T02:23:54.097442: step 236, loss 0.56031, acc 0.78125, learning_rate 0.00271773
2019-01-29T02:23:54.187579: step 237, loss 0.346636, acc 0.890625, learning_rate 0.00271076
2019-01-29T02:23:54.315091: step 238, loss 0.450873, acc 0.8125, learning_rate 0.0027038
2019-01-29T02:23:54.438560: step 239, loss 0.381205, acc 0.890625, learning_rate 0.00269686
2019-01-29T02:23:54.564781: step 240, loss 0.42063, acc 0.828125, learning_rate 0.0026899

2019-01-29T02:24:04.405965: step 320, loss 0.298715, acc 0.921875, learning_rate 0.0021922
2019-01-29T02:24:04.522702: step 321, loss 0.251153, acc 0.9375, learning_rate 0.00218662
2019-01-29T02:24:04.644314: step 322, loss 0.270959, acc 0.953125, learning_rate 0.00218106
2019-01-29T02:24:04.771196: step 323, loss 0.237219, acc 0.96875, learning_rate 0.00217552
2019-01-29T02:24:04.895456: step 324, loss 0.220145, acc 0.96875, learning_rate 0.00216999
2019-01-29T02:24:05.021726: step 325, loss 0.305169, acc 0.921875, learning_rate 0.00216447
2019-01-29T02:24:05.152231: step 326, loss 0.257592, acc 0.96875, learning_rate 0.00215897
2019-01-29T02:24:05.281962: step 327, loss 0.266631, acc 0.953125, learning_rate 0.00215349
2019-01-29T02:24:05.409909: step 328, loss 0.190816, acc 1, learning_rate 0.00214802
2019-01-29T02:24:05.542903: step 329, loss 0.195713, acc 1, learning_rate 0.00214256
2019-01-29T02:24:05.668480: step 330, loss 0.353765, acc 0.90625, learning_rate 0.00213712
2019-01-2

...

2019-01-29T02:32:06.198197: step 4305, loss 0.0618779, acc 1, learning_rate 0.000100051
2019-01-29T02:32:06.326504: step 4306, loss 0.0570975, acc 1, learning_rate 0.00010005
2019-01-29T02:32:06.416420: step 4307, loss 0.0649861, acc 1, learning_rate 0.00010005
2019-01-29T02:32:06.549168: step 4308, loss 0.0792641, acc 0.984375, learning_rate 0.00010005
2019-01-29T02:32:06.636762: step 4309, loss 0.0595093, acc 1, learning_rate 0.00010005
2019-01-29T02:32:06.759985: step 4310, loss 0.0891253, acc 0.984375, learning_rate 0.00010005
2019-01-29T02:32:06.848701: step 4311, loss 0.070132, acc 1, learning_rate 0.00010005
2019-01-29T02:32:06.976228: step 4312, loss 0.0907589, acc 0.984375, learning_rate 0.00010005
2019-01-29T02:32:07.059621: step 4313, loss 0.0792364, acc 1, learning_rate 0.000100049
2019-01-29T02:32:07.185171: step 4314, loss 0.0670692, acc 1, learning_rate 0.000100049
2019-01-29T02:32:07.306476: step 4315, loss 0.0586087, acc 1, learning_rate 0.000100049
2019-01-29T02:32:07

2019-01-29T02:32:17.235912: step 4399, loss 0.0514672, acc 1, learning_rate 0.000100039
2019-01-29T02:32:17.361660: step 4400, loss 0.0863845, acc 1, learning_rate 0.000100039

Evaluation:
2019-01-29T02:32:17.388711: step 4400, loss 0.67599, acc 0.752345

Saved model checkpoint to /root/My_Text_Classification_v3/runs/1548728574/checkpoints/model-4400

2019-01-29T02:32:17.725637: step 4401, loss 0.0670217, acc 1, learning_rate 0.000100039
2019-01-29T02:32:17.815410: step 4402, loss 0.0513977, acc 1, learning_rate 0.000100039
2019-01-29T02:32:17.939460: step 4403, loss 0.0596236, acc 1, learning_rate 0.000100039
2019-01-29T02:32:18.071654: step 4404, loss 0.0924939, acc 0.984375, learning_rate 0.000100039
2019-01-29T02:32:18.198876: step 4405, loss 0.0627389, acc 1, learning_rate 0.000100039
2019-01-29T02:32:18.321845: step 4406, loss 0.0733346, acc 1, learning_rate 0.000100039
2019-01-29T02:32:18.442687: step 4407, loss 0.0792355, acc 1, learning_rate 0.000100038
2019-01-29T02:32:18.569

2019-01-29T02:32:28.377602: step 4490, loss 0.0711091, acc 1, learning_rate 0.000100031
2019-01-29T02:32:28.507523: step 4491, loss 0.0712087, acc 0.984375, learning_rate 0.000100031
2019-01-29T02:32:28.597422: step 4492, loss 0.0629182, acc 0.984375, learning_rate 0.000100031
2019-01-29T02:32:28.713575: step 4493, loss 0.0561833, acc 1, learning_rate 0.000100031
2019-01-29T02:32:28.843968: step 4494, loss 0.0701849, acc 0.984375, learning_rate 0.000100031
2019-01-29T02:32:28.969829: step 4495, loss 0.0693953, acc 0.984375, learning_rate 0.00010003
2019-01-29T02:32:29.094083: step 4496, loss 0.0672024, acc 1, learning_rate 0.00010003
2019-01-29T02:32:29.221711: step 4497, loss 0.0992232, acc 0.984375, learning_rate 0.00010003
2019-01-29T02:32:29.349947: step 4498, loss 0.0529118, acc 1, learning_rate 0.00010003
2019-01-29T02:32:29.477980: step 4499, loss 0.0814118, acc 0.984375, learning_rate 0.00010003
2019-01-29T02:32:29.605040: step 4500, loss 0.0692148, acc 0.983333, learning_rate 

SystemExit: 

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2969: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
%tb